In [18]:
from huggingface_hub import notebook_login

notebook_login()

In [20]:
from huggingface_hub import login

login(token="hello_world")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [21]:
!pip install transformers
!pip install transformers[torch]
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cpu
!pip install pytorch torchvision torchaudio cpuonly -c pytorch

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cpu
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'pytorch'


In [22]:

from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

pipe = pipeline("text2text-generation", model="google/flan-t5-small")

# Test the pipeline
result = pipe("A step by step recipe to make bolognese pasta:")
print(result)

 # Alternatively, load the model and tokenizer directly
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")

# Prepare input
inputs = tokenizer("A step by step recipe to make bolognese pasta:", return_tensors="pt")

# Generate output
outputs = model.generate(**inputs)

# Decode and print the result
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[{'generated_text': 'Pour a cup of bolognese into a large bowl and add the pasta'}]
['Pour a cup of bolognese into a large bowl and add the pasta']


In [23]:
!pip install peft
from peft import LoraConfig, TaskType

peft_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1
)

In [24]:
from peft import get_peft_model

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 344,064 || all params: 77,305,216 || trainable%: 0.4451


In [25]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="NeuraFusionAI/t5-small-lora",
    learning_rate=1e-3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [4]:
!pip install --upgrade datasets

!pip show pyarrow
!pip show datasets
!pip uninstall -y pyarrow
!pip install pyarrow --no-cache-dir
!pip install pyarrow
!pip install transformers
!pip install --upgrade datasets
!pip install datasets
! pip install pyarrow
! pip install pyarrow.parquet
!pip install transformers

# Import necessary libraries
from datasets import load_dataset
import pyarrow.parquet as pq
import pyarrow.lib as lib
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

# Load the dataset
dataset = load_dataset("financial_phrasebank", "sentences_allagree")

# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['sentence'], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Prepare the model
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)

# Set up training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)



Name: pyarrow
Version: 17.0.0
Summary: Python library for Apache Arrow
Home-page: https://arrow.apache.org/
Author: 
Author-email: 
License: Apache Software License
Location: /usr/local/lib/python3.10/dist-packages
Requires: numpy
Required-by: bigframes, cudf-cu12, datasets, db-dtypes, ibis-framework, pandas-gbq, tensorflow-datasets
Name: datasets
Version: 2.21.0
Summary: HuggingFace community-driven open-source library of datasets
Home-page: https://github.com/huggingface/datasets
Author: HuggingFace Inc.
Author-email: thomas@huggingface.co
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, dill, filelock, fsspec, huggingface-hub, multiprocess, numpy, packaging, pandas, pyarrow, pyyaml, requests, tqdm, xxhash
Required-by: 
Found existing installation: pyarrow 17.0.0
Uninstalling pyarrow-17.0.0:
  Successfully uninstalled pyarrow-17.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 183.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver 

ERROR: Could not find a version that satisfies the requirement pyarrow.parquet (from versions: none)
ERROR: No matching distribution found for pyarrow.parquet


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [5]:
model.save_pretrained("output_dir")
model.push_to_hub("NeuraFusionAI/t5-small-lora")

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/NeuraFusionAI/t5-small-lora/commit/341a3af99e33d816316bbda7c65f04c9dc609f00', commit_message='Upload BertForSequenceClassification', commit_description='', oid='341a3af99e33d816316bbda7c65f04c9dc609f00', pr_url=None, pr_revision=None, pr_num=None)

In [8]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from peft import LoraConfig, get_peft_model, TaskType

# Load the base model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")

# Define the LoRA configuration
peft_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1
)

# Apply LoRA to the model
model = get_peft_model(model, peft_config)

# Save the adapted model
model.save_pretrained("output_dir_lora")

# Push the model to the Hugging Face Hub
model.push_to_hub("NeuraFusionAI/t5-small-lora")
tokenizer.push_to_hub("NeuraFusionAI/t5-small-lora")


README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/NeuraFusionAI/t5-small-lora/commit/d10b91d6aa71cd9bdaba9581ec0b68d8504085da', commit_message='Upload tokenizer', commit_description='', oid='d10b91d6aa71cd9bdaba9581ec0b68d8504085da', pr_url=None, pr_revision=None, pr_num=None)